<a href="https://colab.research.google.com/github/Bipolar-Ochka/AnimeBot/blob/master/test_stand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Импорты


In [1]:
import numpy as np
from scipy.linalg import hadamard




---



# 2. Вспомогательные функции

## 2.1. Текст в выбранной кодировке -> массив выбранного типа

По умолчанию: кодировка - **windows-1251**, тип выходных переменных - **bool**. Последнее для удобства маскирования матрицы преобразования.

In [ ]:
def text2bin(text, encoding = 'windows-1251'):
    res = []
    for num in text.encode(encoding):
        res.extend(i for i in map(int, '{:08b}'.format(num)))
    
    return res

def gettextarray(text, encoding = 'windows-1251', type = 'bool'):
    result = np.array(text2bin(text, encoding), dtype=type)
    return (result, result.shape[0])

## 2.2. Битовые операции для разных нумераций матрицы преобразования

In [ ]:
#число -> код грея
def b2gr(n):
    return n ^ (n >> 1)

#код грея -> число
def gr2b(n):
    temp = n
    n >>= 1
    while n != 0:
        temp ^= n
        n >>= 1
        
    return temp

#биты в обратном порядке
def reverse(n, bits):
    b = format(n, 'b')
    b = b.rjust(bits, '0')
    return int(b[::-1],2)

#нумерация по адамару -> по уолшу
def had2wal(val, bits):
    r = reverse(val, bits)
    return gr2b(r)

#нумерация по уолшу -> по пэли
def wal2pal(val):
    return b2gr(val)

## 2.3. Вычисление матрицы преобразования заданного размера

Использовать функцию - get_ort_matrix. Первый параметр - длина массива битов кодируемого сообщения, второй - тип нумерации: 'had' - по Адамару, 'wal'  - по Уолшу, остальное - по Пэли. Возвращает матрицу ортогонального преобразования размера $⌈\log_2 N⌉$

In [ ]:
def get_hadamar(size):
    return hadamard(size)

def get_walsh(size):
    bits = int(np.log2(size))
    had_mat = get_hadamar(size)
    map_wal = {i:had2wal(i, bits) for i in range(size)}
    w = np.zeros((size,size))
    for had, wal in map_wal.items():
        w[wal] = had_mat[had]
    return w

def get_paley(size):
    walsh = get_walsh(size)
    map_pal = {i:wal2pal(i) for i in range(size)}
    p = np.zeros((size,size))
    for wal, pal in map_pal.items():
        p[pal] = walsh[wal]
    return p

def get_ort_matrix(msg_len, type='pal'):
    size = np.ceil(np.log2(msg_len))
    total_size = int(2**size)

    if type == 'had':
        return (total_size, get_hadamar(total_size))
    elif type == 'wal':
        return (total_size, get_walsh(total_size))
    else:
        return (total_size, get_paley(total_size))

## 2.4. Задание соответствия i-му биту массива сообщений -> строки матрицы преобразования

Вход - длина массива битов сообщения. По умолчанию - i-ый бит сообщения соответствует i-ой строке матрицы преобразования

In [ ]:
def get_default_map(msg_len):
    return np.array([i for i in range(msg_len)], dtype='int')

## 2.5. Вычисление суммарного сигнала соответствующего массиву битов сообщения

Использовать - get_sum_function_for_message. Параметры - полная матрица преобразования, массив битов, соответствие из [2.4.](https://colab.research.google.com/drive/1i-hgvaiHeT_Z5MM7c-SuDrc3TxKRR-c2#scrollTo=fV20kWHF2kOH&line=1&uniqifier=1), сдвиг по строкам, сдвиг по столбцам

In [ ]:
def get_walsh_part(arr, msg_len, offset_fun = 0, offset_elem = 0):
    np_arr = np.array(arr, dtype='int')
    
    if(np_arr.ndim != 2):
        raise ValueError('Not 2 dim matrix')

    if(np_arr.shape[0] < msg_len):
        raise ValueError('Not enough functions or function elements')

    return arr[offset_fun : msg_len+offset_fun, offset_elem : msg_len + offset_elem]

def get_mapped_walsh(part_walsh, mapping):
    if(part_walsh.shape[0] < len(mapping)):
        raise ValueError('Matrix have lesser function than requested')
    return np.take(part_walsh, mapping, axis=0)

def get_walsh_masked_for_message(w_matrix, msg_bits, w_fun_mapping , offset_fun = 0, offset_elem = 0):
    msg_len = len(msg_bits)
    full_part_matrix = get_walsh_part(w_matrix, msg_len, offset_fun, offset_elem)
    part_matrix = get_mapped_walsh(full_part_matrix, w_fun_mapping)
    mask = np.array(msg_bits, dtype='bool')
    return full_part_matrix, part_matrix[mask, :]

def get_sum_function_for_message(w_matrix, msg_bits, w_fun_mapping , offset_fun = 0, offset_elem = 0):
    full_part_matrix, mtrx = get_walsh_masked_for_message(w_matrix, msg_bits, w_fun_mapping , offset_fun, offset_elem)
    return (full_part_matrix, np.sum(mtrx, axis=0))

## 2.6. Суммарный сигнал -> Массив битов сообщения

In [ ]:
def decode_sum_fun(part_matrix, sum_fun):
    return np.matmul(part_matrix, sum_fun) / part_matrix.shape[1]